In [ ]:
%pip install transformers==4.31.0 peft==0.4.0 accelerate==0.21.0 datasets==2.14.1 -q

In [ ]:
import os
import argparse

from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    set_seed,
    default_data_collator,
    Trainer,
    TrainingArguments,
)
import torch

In [ ]:
import argparse
parser = argparse.ArgumentParser()

# add model id and dataset path argument
parser.add_argument(
    "--model_id",
    type=str,
    default="NousResearch/Llama-2-7b-hf", # not gated
    help="Model id to use for training.",
)
parser.add_argument(
    "--dataset_path", 
    type=str, 
    default="lm_dataset", 
    help="Path to dataset."
)

# add training hyperparameters for epochs, batch size, learning rate, and seed
parser.add_argument(
    "--epochs", 
    type=int, 
    default=1, 
    help="Number of epochs to train for."
)
parser.add_argument(
    "--per_device_train_batch_size",
    type=int,
    default=1,
    help="Batch size to use for training.",
)
parser.add_argument(
    "--lr", 
    type=float, 
    default=5e-5, 
    help="Learning rate to use for training."
)
parser.add_argument(
    "--seed", 
    type=int, 
    default=42, 
    help="Seed to use for training."
)
parser.add_argument(
    "--gradient_checkpointing",
    type=bool,
    default=True,
    help="Path to deepspeed config file.",
)
parser.add_argument(
    "--bf16",
    type=bool,
    default=True if torch.cuda.get_device_capability()[0] >= 8 else False,
    help="Whether to use bf16.",
)
args, _ = parser.parse_known_args()

## Load dataset

In [ ]:
set_seed(args.seed)

from datasets import load_dataset
from random import randrange

# Load dataset from the hub
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")
dataset = dataset.select(range(1000))

print(f"dataset size: {len(dataset)}")
print(dataset[randrange(len(dataset))])

In [ ]:
def format_dolly(sample):
    instruction = f"### Instruction\n{sample['instruction']}"
    context = f"### Context\n{sample['context']}" if len(sample["context"]) > 0 else None
    response = f"### Answer\n{sample['response']}"
    # join all the parts together
    prompt = "\n\n".join([i for i in [instruction, context, response] if i is not None])
    return prompt

In [ ]:
from transformers import AutoTokenizer

model_id = "NousResearch/Llama-2-7b-hf" # not gated
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from random import randint
from itertools import chain
from functools import partial


# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_dolly(sample)}{tokenizer.eos_token}"
    return sample


# apply prompt template per sample
dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))
# print random sample
print(dataset[randint(0, len(dataset))]["text"])

# empty list to save remainder from batches to use in next batch
remainder = {"input_ids": [], "attention_mask": [], "token_type_ids": []}

def chunk(sample, chunk_length=2048):
    # define global remainder variable to save remainder from batches to use in next batch
    global remainder
    # Concatenate all texts and add remainder from previous batch
    concatenated_examples = {k: list(chain(*sample[k])) for k in sample.keys()}
    concatenated_examples = {k: remainder[k] + concatenated_examples[k] for k in concatenated_examples.keys()}
    # get total number of tokens for batch
    batch_total_length = len(concatenated_examples[list(sample.keys())[0]])

    # get max number of chunks for batch
    if batch_total_length >= chunk_length:
        batch_chunk_length = (batch_total_length // chunk_length) * chunk_length

    # Split by chunks of max_len.
    result = {
        k: [t[i : i + chunk_length] for i in range(0, batch_chunk_length, chunk_length)]
        for k, t in concatenated_examples.items()
    }
    # add remainder to global variable for next batch
    remainder = {k: concatenated_examples[k][batch_chunk_length:] for k in concatenated_examples.keys()}
    # prepare labels
    result["labels"] = result["input_ids"].copy()
    return result


# tokenize and chunk dataset
lm_dataset = dataset.map(
    lambda sample: tokenizer(sample["text"]), batched=True, remove_columns=list(dataset.features)
).map(
    partial(chunk, chunk_length=2048),
    batched=True,
)

# Print total number of samples
print(f"Total number of samples: {len(lm_dataset)}")

## Load model and training

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    args.model_id,
    use_cache=False
    if args.gradient_checkpointing
    else True,
    device_map="auto",
)

In [ ]:
output_dir = "./tmp/llama2"
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=args.per_device_train_batch_size,
    bf16=args.bf16,  # Use BF16 if available
    learning_rate=args.lr,
    num_train_epochs=args.epochs,
    gradient_checkpointing=args.gradient_checkpointing,
    # logging strategies
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="no",
)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset,
    data_collator=default_data_collator,
)

# Start training
trainer.train()

In [ ]:
sagemaker_save_dir="./llama2_dolly"

trainer.model.save_pretrained(
    sagemaker_save_dir, safe_serialization=True
)

# save tokenizer for easy inference
tokenizer = AutoTokenizer.from_pretrained(args.model_id)
tokenizer.save_pretrained(sagemaker_save_dir)